In [17]:
import pandas as pd
import sqlite3
import os 
import numpy as np
"""Connect to the sql database"""
database_path = "database.sqlite"
assert(os.path.exists(database_path)), "this notebook requires create_sql...py to have been run"
db = sqlite3.connect(database_path)
cur = db.cursor()
"""Read to dataframe"""
df = pd.read_sql("""SELECT * 
                FROM CRIMES
                WHERE "LSOA name" LIKE 'Barnet%'
                AND "Crime type" LIKE 'Burglary'""", con=db)
df

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,c2ccc34ed8f7e6c4529a2ee6464019d189970506889264...,2020-04,Metropolitan Police Service,Metropolitan Police Service,-0.210051,51.657237,On or near Wentworth Road,E01000249,Barnet 001B,Burglary,Investigation complete; no suspect identified,None
1,bb5f9f0e9adf955886f1884e3eefc04cff57ac50f0b7da...,2020-04,Metropolitan Police Service,Metropolitan Police Service,-0.203549,51.645780,On or near Supermarket,E01000292,Barnet 001E,Burglary,Investigation complete; no suspect identified,None
2,0898a274117379acf52b279d2eaf103ede17ff9020a674...,2020-04,Metropolitan Police Service,Metropolitan Police Service,-0.203549,51.645780,On or near Supermarket,E01000292,Barnet 001E,Burglary,Investigation complete; no suspect identified,None
3,aaf394e5947f810a266a6e7dbc4b322206f92fbffe6aa5...,2020-04,Metropolitan Police Service,Metropolitan Police Service,-0.203549,51.645780,On or near Supermarket,E01000292,Barnet 001E,Burglary,Investigation complete; no suspect identified,None
4,5feed401361e5c82e326e963773b22ae741369e2634b57...,2020-04,Metropolitan Police Service,Metropolitan Police Service,-0.205051,51.652412,On or near Cattley Close,E01000292,Barnet 001E,Burglary,Investigation complete; no suspect identified,None
...,...,...,...,...,...,...,...,...,...,...,...,...
7452,fda0870be993d2c9352ae14a540bf529c15a7aa9d92a4e...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.202163,51.559100,On or near Hocroft Road,E01000139,Barnet 041B,Burglary,Under investigation,None
7453,c6c2d9a772695865f4ca33fce59c26c48fe3a1cdac8c22...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.202163,51.559100,On or near Hocroft Road,E01000139,Barnet 041B,Burglary,Under investigation,None
7454,e5bab2bd4f31de44515dbf125b85284392d0349139a1d3...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.197843,51.561093,On or near Church Walk,E01000140,Barnet 041C,Burglary,Under investigation,None
7455,823465427f8764e250a48312715a4bd87e8012412046c6...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.199449,51.563896,On or near Llanelly Road,E01000140,Barnet 041C,Burglary,Under investigation,None


In [18]:
LSOA_list = df["LSOA code"].unique()

df_LSOA_per_month = pd.DataFrame(LSOA_list,columns=["LSOA"]).set_index("LSOA")
df_LSOA_per_month

""
LSOA
E01000249
E01000292
E01000251
E01000172
E01000268
...
E01000295
E01000167
E01000288


In [19]:
months = np.array(df["Month"].unique())
months.sort()
for cur_month in months:
    df_month  = df[df["Month"] == cur_month]
    
    # prevent counting same crime id twice
    df_month = df_month[["LSOA code", "Crime ID"]].drop_duplicates().groupby("LSOA code").count()
    df_month = df_month.reindex(LSOA_list).fillna(0)
    df_month = df_month.rename(columns={"Crime ID": cur_month})
    
    df_LSOA_per_month[cur_month] = df_month
    


df_LSOA_per_month = df_LSOA_per_month.reindex(sorted(df_LSOA_per_month.columns), axis=1).sort_index()
df_LSOA_per_month

,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,...,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03
LSOA,,,,,,,,,,,,,,,,,,,,,
E01000115,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,3.0
E01000116,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,2.0,1.0
E01000117,1.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
E01000118,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,0.0,1.0,...,0.0,1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,2.0
E01000119,1.0,0.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,2.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E01000322,2.0,0.0,0.0,0.0,0.0,4.0,3.0,5.0,1.0,0.0,...,0.0,2.0,0.0,1.0,0.0,2.0,2.0,0.0,2.0,1.0
E01000323,1.0,0.0,1.0,2.0,0.0,1.0,0.0,2.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0
E01000324,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.0,1.0,0.0,0.0


In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster

In [21]:
# example
# Download data
# ==============================================================================
url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv')
data = pd.read_csv(url, sep=',', header=0, names=['y', 'datetime'])

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y/%m/%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data['y']
data = data.sort_index()

# Data partition train-test
# ==============================================================================
end_train = '2005-06-01 23:59:00'
print(f"Train dates : {data.index.min()} --- {data.loc[:end_train].index.max()}  (n={len(data.loc[:end_train])})")
print(f"Test dates  : {data.loc[end_train:].index.min()} --- {data.index.max()}  (n={len(data.loc[end_train:])})")


Train dates : 1991-07-01 00:00:00 --- 2005-06-01 00:00:00  (n=168)
Test dates  : 2005-07-01 00:00:00 --- 2008-06-01 00:00:00  (n=36)


In [22]:
XY_df = df_LSOA_per_month.T
XY_df

LSOA,E01000115,E01000116,E01000117,E01000118,E01000119,E01000120,E01000121,E01000122,E01000123,E01000124,...,E01000317,E01000318,E01000319,E01000320,E01000321,E01000322,E01000323,E01000324,E01033572,E01033573
2020-04,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,3.0,0.0
2020-05,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,5.0
2020-06,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,3.0,0.0,1.0,2.0,3.0,0.0
2020-07,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,...,0.0,0.0,0.0,2.0,3.0,0.0,2.0,1.0,0.0,1.0
2020-08,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0,4.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
2020-09,2.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.0,2.0,0.0,3.0,3.0,4.0,1.0,0.0,6.0,3.0
2020-10,0.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,0.0,...,1.0,0.0,1.0,2.0,1.0,3.0,0.0,0.0,5.0,3.0
2020-11,0.0,1.0,0.0,3.0,1.0,2.0,0.0,1.0,0.0,0.0,...,2.0,3.0,2.0,3.0,2.0,5.0,2.0,1.0,5.0,8.0
2020-12,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,3.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0
2021-01,1.0,1.0,0.0,1.0,2.0,1.0,2.0,2.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,3.0


In [23]:
D = XY_df.to_numpy()
D

array([[0., 0., 1., ..., 0., 3., 0.],
       [0., 0., 0., ..., 0., 3., 5.],
       [1., 1., 0., ..., 2., 3., 0.],
       ...,
       [1., 0., 1., ..., 1., 1., 1.],
       [0., 2., 1., ..., 0., 4., 0.],
       [3., 1., 1., ..., 0., 4., 0.]])

In [24]:
n_neighborhoods = D.shape[1]  # Number of neighborhoods
n_months = D.shape[0]  # Number of months

In [26]:
import statsmodels.api as sm

train_data = D[:n_months-1, :]
test_data = D[n_months-1, :]

# Fit the VAR model
model = sm.tsa.VAR(train_data)
model_fit = model.fit()

# Predict the next month's events for each neighborhood
prediction = model_fit.forecast(train_data, steps=1)
np.sum((test_data - prediction)**2)

334.38357192714